 <div align="center"><h1> ANALYSE DES CLASSEMENTS DE PRODUITS </h1></div>

Note : *Le code source est disponible [ici](https://github.com/toshiro10/LangageScript)*

## CAS DU MAGAZINE 60 MILLIONS DE CONSOMMATEURS - Couches-culottes

Le magazine 60 millions de consommateurs dans son numéro 540 du mois d’Août 2018 a publié, après des essais comparatifs, un classement de plusieurs marques de couches-culottes pour enfants. Les résultats de leur enquête sont donnés
par le tableau ci-dessous.


<img src="../docs/tableau.png">

Dans cette section, nous essayerons d’analyser ce classement élaboré à partir de deux critères : la performance et la
composition avec des poids respectifs de 60% et 40 %.

## 1. Le score global (/20) associé à chaque produit peut-il être expliqué par une somme pondérée ?

A la lecture du classement publié, il semble clair, pour un consommateur, que le score global donné à chaque produit
résulte d’une moyenne pondérée (somme pondérée) de ses évaluations sur les deux critères performance et composition.
L’objectif dans cette partie est de vérifier l’exactitude ou non de cette hypothèse.

Pour cela, nous allons associer un nombre réel positif à chaque évaluation qualitative attribuée à un produit. Ainsi, pour
le produit “Joone” représenté par la lettre a sur le Tableau 1, si son score de 17/20 est obtenu à partir d’une somme
pondérée notée f, alors il existe des nombres réels positifs U1a(+++) ∈ [17, 20] et U2a(+++) ∈ [17, 20] tels que

**<p style="text-align: center;">f(a) = 0.6 U1a(+++) + 0.4 U2a(+++) = 17</p>**

Plus généralement, pour un produit x dont le score global f(x) est donné par une somme pondérée f, si on note Uix(αix)
le nombre réel associé à l’évaluation qualitative αix du produit x sur le critère i (la valeur qualitative αix étant donnée
par le magazine), alors on aura


**<p style="text-align: center;">f(x) = 0.6 U1x(α1x) + 0.4 U2x(α2x)</p>**

Pour déterminer si le classement des couches-culottes, fourni par le magazine, est compatible avec un modèle d’évaluation
basé sur la somme pondérée, il nous suffit alors de résoudre le programme linéaire PL1 suivant (un système d’inéquations
linéaires est suffisant pour ce test) :

<img src="../docs/pl.png">

où
* La fonction objectif donnée par l’équation (3) consiste, ici, à maximiser le nombre réel U1a(+++) associé à l’évaluation de la couche-culotte “Joone” sur le critère Performance. En réalité, étant donné que nous cherchons simplement à satisfaire les contraintes de ce programme linéaire, le choix de n’importe quelle fonction objectif suffira à résoudre notre problème. En clair le choix de la fonction objectif de PL1 est arbitraire.


* Les contraintes (4) à (15) correspondent à l’écriture sous forme de somme pondérée, comme dans la formule 2 ci-dessus, des 12 produits à évaluer.


* Les contraintes (16) à (19) correspondent à l’ordre établi dans le classement des 12 produits du magazine. A noter que la valeur 0.1, dans certains de ces contraintes, permet simplement de satisfaire une inégalité stricte, en termesde notes globales, entre certains produits.

* Les contraintes (20) à (29) permettent d’assurer l’appartenance de chaque nombre réel Uix(αix) à un intervalle lié à la note qualitative αix telle que donnée par le magazine.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)   

 ### 1.1 fonction CheckAdditiveModel

In [2]:
from system import checkAdditiveModel, createUpdateModel, compareRankings
from optlang import Variable
from loader import getOriginalData, exportInExcel
from IPython.display import display, display_html, HTML
from json2html import *
from IPython.display import HTML

**CheckAdditiveModel** retourne un message d’erreur si le classement n’est pas explicable par un modèle type
somme pondérée (dans ce cas, le programme linéaire ou système linéaire correspondant n’admet pas de solution
réalisable). Au cas où cela est possible, le programme retournera alors des valeurs numériques à associer à chaque
produit (pour chaque critère d’évaluation), ainsi que la valeur de la somme pondérée qui lui correspond, dans ce notebook et sous fichier **Excel** (répertoire **data_solved**)

La fonction est construite pour être générique et sera appelée sur différentes propositions de résolution du problème

 ## 1.2 
 ### 1.2.1 Test de la fonction avec le classement des couches-culottes

In [3]:
csv_name='../data/data_couches_original.xlsx'
model_name_1 = 'Programme Lineaire - Classement Couches-culottes avec notes' 

La résolution du problème passe par une formalisation en Programme Linéaire

In [4]:
result_2_1 = checkAdditiveModel(csv_name=csv_name,
                      model_name=model_name_1,
                      eval_expr='x1',
                      direction='max',
                      with_scores=True)
print(result_2_1)

Le Programme Lineaire - Classement Couches-culottes avec notes 
 du fichier ../data/data_couches_original.xlsx n'admet pas de solution


#### Commentaires

On s'aperçoit qu'il n'y a pas de solution réalisable. Le classement présenté par le magazine n'est donc pas - avec les informations à notre disposition - explicable par un modèle de somme pondérée.


 ### 1.2.2 CheckAdditiveModel avec le classement des couches lorsque la note globale de chaque couche-culotte n’est pas fixée

In [5]:
model_name_2 = 'Programme Lineaire - Classement Couches-culottes sans note'

In [6]:
result_2_2 = checkAdditiveModel(csv_name=csv_name,
                  model_name=model_name_2,
                  eval_expr='x1',
                  direction='max')
print(result_2_2)

Le Programme Lineaire - Classement Couches-culottes sans note 
 du fichier ../data/data_couches_original.xlsx n'admet pas de solution


En prenant des scores quelconques entre 0 et 20 au lieu de prendre les scores fixés par le magazine on constate que le programme linéaire reste toujours non réalisable. Et donc qu'il n'existe pas un modèle de somme pondérée satisfaisant tous les critères énoncés par le magazine.

 ### 1.2.3 Commentaires

La compréhension intuitive des résultats, suppose qu'il existe deux critères de classement : Performance et composition. La composition compte pour 40%, et la performance pour 60%.

Or, en modélisant ce classement, avec ces coefficients liés à la performance et à la composition, on constate qu'il n'existe pas de solution réalisable exacte de ce problème via un modèle de somme pondérée.

Ces résultats mettent en cause au moins la présentation de cette étude.


# 1.3

Dans cette question, nous allons supposer que l’évaluation globale n’est pas la même pour les couches-culottes
“Naty” et “Pamp. Activ” d’une part, et les couches-culottes “Love & Green” et “Lotus Baby” d’autre part.

Pour cela, nous allons supprimer du programme PL1 les deux contraintes suivantes : **fd = fe** de l’équation (17)
et **fi = fj** de l’équation (18). 

Nous allons également supprimer du programme PL1 la contrainte **17 < U2c(+++)**,
liée à la couche c-Pamp.Baby.

## 1.3.1 Le meilleur et le pire score global obtenu par le produit “Joone”

#### Meilleur score

In [7]:
update_model_Q3 = createUpdateModel([Variable ('x6', ub = 20)], ['c16', 'c21'])

In [8]:
result_original = getOriginalData(csv_name)
model_name_3 = 'Programme Lineaire - Meilleur score Joone'
result_3_1_max = checkAdditiveModel(csv_name=csv_name,
                  model_name=model_name_3,
                  eval_expr='y1',
                  direction='max',
                  update_model=update_model_Q3)

In [9]:
df1_s = result_original.style.set_table_attributes("style='display:inline'").set_caption('Modèle original')
df2_s = result_3_1_max.style.set_table_attributes("style='display:inline'").set_caption(model_name_3)

display_html(df1_s._repr_html_()+df2_s._repr_html_(), raw=True)

,Produit,Performance,Composition,Score
0,a- Joone,+++,+++,17
1,b- Pamp. Prem,++,++,14.5
2,c- Pamp. Baby,+,+++,12.5
3,d- Naty,+,+++,12.5
4,e- Pamp. Activ.,+,+,12.5
5,f- Carref.Baby,++,+,12.5
6,g- Lupilu,++,-,12
7,h- Mots d’enfants,+,-,12
8,i- Love & Green,++,-,9.5
9,j- Lotus Baby,++,--,9.5


#### Pire score

In [10]:
model_name_4 = 'Programme Lineaire - Pire score Joone'
result_3_1_min = checkAdditiveModel(csv_name=csv_name,
                  model_name=model_name_4,
                  eval_expr='y1',
                  direction='min',
                  update_model=update_model_Q3)

In [11]:
df2_s = result_3_1_min.style.set_table_attributes("style='display:inline'").set_caption(model_name_4)
display_html(df1_s._repr_html_()+df2_s._repr_html_(), raw=True)

,Produit,Performance,Composition,Score
0,a- Joone,+++,+++,17
1,b- Pamp. Prem,++,++,14.5
2,c- Pamp. Baby,+,+++,12.5
3,d- Naty,+,+++,12.5
4,e- Pamp. Activ.,+,+,12.5
5,f- Carref.Baby,++,+,12.5
6,g- Lupilu,++,-,12
7,h- Mots d’enfants,+,-,12
8,i- Love & Green,++,-,9.5
9,j- Lotus Baby,++,--,9.5


#### Commentaires

En supposant que les couches "Naty" et "Pamp. Activ", ainsi que les couches "Love & Green" et "Lotus Baby" n'ont plus nécessairement le même score et en supposant également que la note de composition de la couche "Pamp Baby" n'est plus nécessairement supérieure à 17/20, on obtient les résultats suivants :

Le score maximal obtenu pour la couche "Joone" est de 20/20. Le score minimal est de 17/20.

On observe sous ces hypothèses, que la note de la couche "Love & Green" passe de 9.5 à 10.6 soit une progression de 1.1 point.


## 1.3.2 Le meilleur et le pire score global obtenu par le produit “Lillydoo”

#### Meilleur score

In [12]:
model_name_5 = 'Programme Lineaire - Meilleur score Lillydoo'
result_3_2_max = checkAdditiveModel(csv_name=csv_name,
                  model_name=model_name_5,
                  eval_expr='y12',
                  direction='max',
                  update_model=update_model_Q3)

In [13]:
df2_s = result_3_2_max.style.set_table_attributes("style='display:inline'").set_caption(model_name_5)
display_html(df1_s._repr_html_()+df2_s._repr_html_(), raw=True)

,Produit,Performance,Composition,Score
0,a- Joone,+++,+++,17
1,b- Pamp. Prem,++,++,14.5
2,c- Pamp. Baby,+,+++,12.5
3,d- Naty,+,+++,12.5
4,e- Pamp. Activ.,+,+,12.5
5,f- Carref.Baby,++,+,12.5
6,g- Lupilu,++,-,12
7,h- Mots d’enfants,+,-,12
8,i- Love & Green,++,-,9.5
9,j- Lotus Baby,++,--,9.5


#### Pire Score

In [14]:
model_name_6 = 'Programme Lineaire - Pire score Lillydoo'
result_3_2_min = checkAdditiveModel(csv_name=csv_name,
                  model_name='Programme Lineaire - Pire score Lillydoo',
                  eval_expr='y12',
                  direction='min',
                  update_model=update_model_Q3)

In [15]:
df2_s = result_3_2_min.style.set_table_attributes("style='display:inline'").set_caption(model_name_6)
display_html(df1_s._repr_html_()+df2_s._repr_html_(), raw=True)

,Produit,Performance,Composition,Score
0,a- Joone,+++,+++,17
1,b- Pamp. Prem,++,++,14.5
2,c- Pamp. Baby,+,+++,12.5
3,d- Naty,+,+++,12.5
4,e- Pamp. Activ.,+,+,12.5
5,f- Carref.Baby,++,+,12.5
6,g- Lupilu,++,-,12
7,h- Mots d’enfants,+,-,12
8,i- Love & Green,++,-,9.5
9,j- Lotus Baby,++,--,9.5


#### Commentaires

Le pire score pour LillyDoo est 6/20 et le meilleur résultat espéré est 10,1. 
Sous ces hypothèses, en maximisant le score de "LillyDoo", on a "Love & Green", "Pomette" et "illydoo" voient leurs notes passer au-dessus de 10.


## 1.3.3 Conclusions

In [16]:
%%capture
dict_max_y1 = compareRankings(result_original['Score'], result_3_1_max['Score'])
dict_min_y1 = compareRankings(result_original['Score'], result_3_1_min['Score'])
dict_max_y12 = compareRankings(result_original['Score'], result_3_2_max['Score'])
dict_min_y12 = compareRankings(result_original['Score'], result_3_2_min['Score'])

In [17]:
#Coefficients Q3.1
print("Max Score Joone")
display(HTML(json2html.convert(json = dict_max_y1)))
print("Min Score Joone")
display(HTML(json2html.convert(json = dict_min_y1)))
#Coefficients Q3.2
print("Max Score Lillydoo")
display(HTML(json2html.convert(json = dict_max_y12)))
print("Min Score Lillydoo")
display(HTML(json2html.convert(json = dict_min_y12)))

Max Score Joone


coef - Spearman,0.9803024573066307
p - Spearman,2.259433333683426e-08
tau - Kendall,0.9503819266229829
p_val - Kendall,6.317712090483196e-05
Diff Moyenne,-0.375


Min Score Joone


coef - Spearman,0.9803024573066307
p - Spearman,2.259433333683426e-08
tau - Kendall,0.9503819266229829
p_val - Kendall,6.317712090483196e-05
Diff Moyenne,-0.625


Max Score Lillydoo


coef - Spearman,0.9803024573066307
p - Spearman,2.259433333683426e-08
tau - Kendall,0.9503819266229829
p_val - Kendall,6.317712090483196e-05
Diff Moyenne,0.11666666666666477


Min Score Lillydoo


coef - Spearman,0.9803024573066307
p - Spearman,2.259433333683426e-08
tau - Kendall,0.9503819266229829
p_val - Kendall,6.317712090483196e-05
Diff Moyenne,-0.625


Afin de comprendre le processus de notation suivi par le magazine, nous avons observé les scores de toutes les couches en minimisant la note de la pire couche et en la maximisant. 

Les scores globaux changent d’une manière significative, tandis que le classement reste inchangé. Notons que le classement présenté au préalable comprenait une égalité d’une part entre quatre couches culottes qui avaient le score de 12,5 et d’autre part entre trois couches qui avaient le score de 9,5. 

Le relâchement des contraintes dans notre programme linéaire a pour effet de discriminer les couches au niveau des notes mais pas au niveau du classement. 

Nous avons calculé deux tests de corrélation : Spearman et Kendall. Un coefficient de corrélation calcule dans quelle mesure deux variables tendent à changer ensemble. Le coefficient décrit l'importance et le sens de la relation. 
    
Nous avons utilisé ces deux coefficients car ils traitent le cas où la relation n’est pas linéaire. Ces deux coefficients comparent les rangs obtenus selon les notes. 

En relâchant les contraintes d’égalité, nous avons obtenu deux scores différents mais qui gardent le même rang pour chaque couche. D’où l’invariance du coefficient de Spearman et Kendall dans le cas de maximisation et minimisation qui prennent les valeurs respectives de 98% et 95%.


## 1.4

A présent, nous choisissons de ne plus tenir compte de l’ordre des produits fourni dans le classement publié par le
magazine. En d’autres termes, nous supprimons les contraintes **(16)** à **(19)**. Nous allons également supprimer du
programme PL1 la contrainte **17 < U2c(+++)**, liée à la couche **c-Pamp.Baby**.

In [18]:
update_model_Q4 = createUpdateModel([Variable ('x6', ub = 20)], ['c13', 'c14', 'c15', 
                                                                    'c16', 'c17', 'c18',
                                                                    'c19', 'c20', 'c21',
                                                                    'c22', 'c23'])

## 1.4.1 Score global maximal obtenu par chacun des 12 produits

Maintenant nous sommes dans le cas ou on tient plus compte du classement fourni par le magazine. On exécute notre programme linéaire qui cherche la meilleure note espérée par chaque couche et on observe : 

La meilleure note espérée est comprise entre 10,1/20 et 20/20.
Le classement des couches changent (Pamp Activ, Mots d’enfants descendent dans le classement).


In [19]:
%%capture
model_name_7 = 'Programme Lineaire - Score global maximal pour chaque produit'
result_4_1_all_max = checkAdditiveModel(csv_name=csv_name,
                      model_name=model_name_5,
                      eval_expr='all',
                      direction='max',
                      update_model=update_model_Q4)

dict_max_all = compareRankings(result_original['Score'], result_4_1_all_max['Score']) 

In [20]:
df2_s = result_4_1_all_max.style.set_table_attributes("style='display:inline'").set_caption(model_name_7)
display_html(df1_s._repr_html_()+df2_s._repr_html_(), raw=True)

,Produit,Performance,Composition,Score
0,a- Joone,+++,+++,17
1,b- Pamp. Prem,++,++,14.5
2,c- Pamp. Baby,+,+++,12.5
3,d- Naty,+,+++,12.5
4,e- Pamp. Activ.,+,+,12.5
5,f- Carref.Baby,++,+,12.5
6,g- Lupilu,++,-,12
7,h- Mots d’enfants,+,-,12
8,i- Love & Green,++,-,9.5
9,j- Lotus Baby,++,--,9.5


In [21]:
print("Max Score pour toutes les marques")
display(HTML(json2html.convert(json = dict_min_y12)))

Max Score pour toutes les marques


coef - Spearman,0.9803024573066307
p - Spearman,2.259433333683426e-08
tau - Kendall,0.9503819266229829
p_val - Kendall,6.317712090483196e-05
Diff Moyenne,-0.625


## 1.4.2 Conclusion

Nous observons une baisse des coefficients de Spearman et Kendall qui est dû au changement de rang des couches. 

La positivité de ce coefficient indique qu’il existe une corrélation, et que si les scores de départ augment, les scores obtenus augmenteront aussi. 

In [22]:
%%capture
exportInExcel('./results_notebook/Couches_Analyse_Classement.xlsx', 'Q3.1',
                 [result_original, result_3_1_max, result_3_1_min],
                 ['Modèle Original',model_name_3, model_name_4],
                 [dict_max_y1,dict_min_y1])

In [23]:
%%capture
exportInExcel('./results_notebook/Couches_Analyse_Classement.xlsx', 'Q3.2',
                 [result_original, result_3_2_max, result_3_2_min],
                 ['Modèle Original',model_name_5, model_name_6],
                 [dict_max_y12,dict_min_y12])


In [24]:
%%capture
exportInExcel('./results_notebook/Couches_Analyse_Classement.xlsx', 'Q4',
                 [result_original, result_4_1_all_max],
                 ['Modèle Original',model_name_7],
                 [dict_max_all])

# 2 Application d’une démarche de classification multicritère : la méthode ELECTRE TRI

Dans cette partie, au lieu d’attribuer une note (/20) à chaque couche-culotte, nous allons plutôt l’affecter à une classe à l’aide d’une approche ne nécessitant pas une normalisation des critères : La méthode ELECTRE TRI. Cette dernière est une approche d’Aide MultiCritère à la Décision qui vise à résoudre des problèmes d’affectation (classification) d’objets dans des catégories prédéfinies.

Comme indiqué ci-dessus, les poids associés aux deux critères du classement des couches-culottes, Performance et Com- position, sont donnés par le vecteur W = (3/5; 2/5). Nous retenons ici comme catégories ou classes, les 5 catégories suivantes (compatibles avec celles suggérées par le magazine) : ‘Très Bon” (catégorie C5), “Bon” (catégorie C4), “Ac- ceptable” (C3), “Insuffisant” (catégorie C2) et “Inacceptable” (C1).


La méthode ELECTRE-TRI consiste alors à affecter chacune des 12 couches-culottes à une de ces 5 catégories.
On va également supposer que chaque catégorie Ci est délimitée par une frontière supérieure notée bi+1 et une frontière inférieure bi. Les frontières bi+1 et bi sont appelées “profils” et représentent des couches-culottes de référence qui peuvent être fictives. Il y a une dominance paréto-stricte entre bi+1 et bi. Pour ce classement des couches, nous aurons :
• C1 délimitée par b2 et b1 ; 
• C2 délimitée par b3 et b2 ; 
• C3 délimitée par b4 et b3;
• C4 délimitée par b5 et b4;
• C5 délimitée par b6 et b5.

Ainsi, comme l’affectation se fait dans 5 catégories distinctes, le profil b3 représente la frontière entre les classes “Très Bon” et “Bon”, et b2 la frontière entre les classes “Bon” et “Moyen”.

Le principe de la méthode ELECTRE TRI consiste, non pas à comparer les couches-culottes entre elles, mais à les comparer aux six couches-culottes de référence b6, b5, b4, b3, b2 et b1 dont les évaluations sur chaque critère sont résumés à la Table 2. 

Attention : dans ce tableau, les profils b6 et b1 doivent toujours avoir des valeurs extrêmes, ne pouvant jamais être atteintes, sur chaque critère.

Ainsi, l’affectation d’une couche-culotte à une catégorie dépendra de sa comparaison aux profils b6, b5, b4, b3, b2 et b1. Plus formellement, l’affectation d’une couches-culottes dans les catégories se base sur le concept de sur-classement.

<img src="../docs/tableau_2.png">

On dira qu’une couche-culotte H surclasse le profil bi (respectivement le profil bi surclasse la couche-culotte H) et on note H S bi (respectivement bi S H) si H est au moins aussi bon que bi (respectivement bi est au moins aussi bon que H) sur une majorité de les critères (la majorité étant définie par un seuil de majorité λ). Les étapes de la méthode ELECTRE TRI se décrivent comme suit :

<img src="../docs/maths.png">

<img src="../docs/somme.png">

<img src="../docs/question.png">

## 2.5.1 Matrices de concordances partielles et globales

In [25]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path) 

In [26]:
from electre_tri import ConcordancePartielleHbi, ConcordancePartiellebiH, ConcordonceGlobaleHbi,ConcordonceGlobalebiH 
from electre_tri import SurclassementHbi,SurclassementbiH,EvaluationPessimiste,EvaluationOptimiste, compareClassification
from loader import getElectreTriData, exportInExcel
import pandas as pd

### a. Test de la fonction de creation des matrices de concordance Partielles pour les couches culottes

In [27]:
csv_name='../data/data_couches_original.xlsx'
direction="max"
result_original = getElectreTriData(csv_name) 
lambda_2=0.75

vu qu'on a choisi de maximiser l'algorithme va executer la fonction ConcordancePartielHbi et ConcordancebiH selon : 

<img src="../docs/concorde.png">

In [28]:
result_2_5_1Hbi = ConcordancePartielleHbi(csv_name=csv_name,
                      direction=direction)

result_2_5_1biH = ConcordancePartiellebiH(csv_name=csv_name,
                      direction=direction)

######  commentaires

on voit que la fonction  concordanceHbi et concordance biH renvoit une liste de matrice selon les nombre de critères et pour les couches on aura deux matrices , une pour la performance et une pour la composition .

### b. Test de la fonction de creation des matrices de concordance globale pour les couches culottes

pour creer la matrice de concordance globale pour chaque critère de classment de produit on suit la formule suivante : 

<img src="../docs/globale.png">

In [29]:
result_2_5_2Hbi = ConcordonceGlobaleHbi(csv_name=csv_name,
                      direction=direction )

result_2_5_2_biH = ConcordonceGlobalebiH(csv_name=csv_name,
                      direction=direction )

la premiere matrice correspond au matrice  C(H,bi) de concrdance globale pour les deux critères Performance et Composition .

la deuxième matrice correspond au matrice  C(bi,H) de concrdance globale pour les deux critères Performance et Composition 

## 2.5.3 La methode de surclassment des couches culottes 

on dit : 

H surclasse bi et on notera H S bi si et seulement si C(H, bi) ≥ λ.

bi surclasse H et on notera bi S H si et seulement si C(bi, H) ≥ λ.

PS : λ est le seuil de majorité généralement superieure à 50 % 

In [30]:
result_2_5_3Hbi=SurclassementHbi(lamda=lambda_2,
                      csv_name=csv_name,
                      direction=direction)


result_2_5_3biH = SurclassementbiH(lamda=lambda_2,
                     csv_name=csv_name,
                     direction=direction)

## 2.5.4 Phase final : Procedures d'evaluation Optimiste et pessimiste 

### a. Test des procedures EvalOptimiste  et EvalPessimiste ainsi les taux de mauvaise classification(optimiste pessimiste) selon λ = 75 % 

In [31]:
EvalPessimisteResult_lambda_2 = EvaluationPessimiste(lamda=lambda_2,
                     csv_name=csv_name,
                     direction=direction)
EvalOptimisteResult_lambda_2 = EvaluationOptimiste(lamda=lambda_2,
                     csv_name=csv_name,
                     direction=direction)  
taux_mauvaise_classification_opt_lambda_2= compareClassification(TypeEval="optimiste",
                                                       lamda=lambda_2,
                                                       csv_name=csv_name,
                                                       direction=direction)
taux_mauvaise_classification1_pess_lambda_2 = compareClassification(TypeEval="pessimiste",
                                                       lamda=lambda_2,
                                                       csv_name=csv_name,
                                                       direction=direction)


result_original['Eval Pessimiste'] = EvalPessimisteResult_lambda_2
result_original['Eval Optimiste'] = EvalOptimisteResult_lambda_2
display(result_original)

    
print("Taux mauvaise classification Optimiste", taux_mauvaise_classification_opt_lambda_2,'%')
   
print("Taux mauvaise classification Pessimiste", taux_mauvaise_classification1_pess_lambda_2,'%')


df_Eval_Opt_lambda_2=pd.DataFrame(EvalOptimisteResult_lambda_2,columns=["Classement Optimiste"])
df_Eval_Pes_lambda_2=pd.DataFrame(EvalPessimisteResult_lambda_2,columns=["Classement Pessimiste"])


new_df_lambda_2 = pd.concat([result_original, df_Eval_Opt_lambda_2, df_Eval_Pes_lambda_2], axis=1, sort=False)

result_lambda_2 = {"Tx Optimiste": taux_mauvaise_classification_opt_lambda_2, "Tx Pessimiste" :taux_mauvaise_classification1_pess_lambda_2}  
   

,Produit,Note_magazine,Eval Pessimiste,Eval Optimiste
0,a- Joone,C5,Très Bon,Très Bon
1,b- Pamp. Prem,C4,Bon,Bon
2,c- Pamp. Baby,C3,Acceptable,Bon
3,d- Naty,C3,Acceptable,Bon
4,e- Pamp. Activ.,C3,Acceptable,Acceptable
5,f- Carref.Baby,C3,Acceptable,Acceptable
6,g- Lupilu,C3,Insuffisant,Acceptable
7,h- Mots d’enfants,C3,Insuffisant,Insuffisant
8,i- Love & Green,C2,Insuffisant,Acceptable
9,j- Lotus Baby,C2,Très Insuffisant,Acceptable


Taux mauvaise classification Optimiste 58.333333333333336 %
Taux mauvaise classification Pessimiste 33.33333333333333 %


### b. Test des procedures EvalOptimiste  et EvalPessimiste ainsi les taux de mauvaise classification(optimiste pessimiste) selon λ = 55 % 

In [32]:
lambda_1 = 0.55
result_2_5_3Hbi=SurclassementHbi(lamda=lambda_1,
                      csv_name=csv_name,
                      direction=direction)


result_2_5_3biH = SurclassementbiH(lamda=lambda_1,
                     csv_name=csv_name,
                     direction=direction)

EvalPessimisteResult_lambda_1 = EvaluationPessimiste(lamda=lambda_1,
                     csv_name=csv_name,
                     direction=direction)
EvalOptimisteResult_lambda_1 = EvaluationOptimiste(lamda=lambda_1,
                     csv_name=csv_name,
                     direction=direction)  

taux_mauvaise_classification_opt_lambda_1= compareClassification(TypeEval="optimiste",
                                                       lamda=lambda_1,
                                                       csv_name=csv_name,
                                                       direction=direction)
taux_mauvaise_classification1_pess_lambda_1 = compareClassification(TypeEval="pessimiste",
                                                       lamda=lambda_1,
                                                       csv_name=csv_name,
                                                       direction=direction)


result_original['Eval Pessimiste'] = EvalPessimisteResult_lambda_1
result_original['Eval Optimiste'] = EvalOptimisteResult_lambda_1
display(result_original)
   
   
   
print("Taux mauvaise classification Opt", taux_mauvaise_classification_opt_lambda_1,'%')
   
print("Taux mauvaise classification Pessimiste", taux_mauvaise_classification1_pess_lambda_1,'%')

df_Eval_Opt_lambda1=pd.DataFrame(EvalOptimisteResult_lambda_1,columns=["Classement Optimiste"])
df_Eval_Pes_lambda1=pd.DataFrame(EvalPessimisteResult_lambda_1,columns=["Classement Pessimiste"])
   
new_df_lambda1 = pd.concat([result_original, df_Eval_Opt_lambda1, df_Eval_Pes_lambda1], axis=1, sort=False)

result_lambda1 = {"Tx Optimiste": taux_mauvaise_classification_opt_lambda_1, "Tx Pessimiste" :taux_mauvaise_classification1_pess_lambda_1  }


,Produit,Note_magazine,Eval Pessimiste,Eval Optimiste
0,a- Joone,C5,Très Bon,Très Bon
1,b- Pamp. Prem,C4,Bon,Bon
2,c- Pamp. Baby,C3,Acceptable,Acceptable
3,d- Naty,C3,Acceptable,Acceptable
4,e- Pamp. Activ.,C3,Acceptable,Acceptable
5,f- Carref.Baby,C3,Bon,Bon
6,g- Lupilu,C3,Bon,Bon
7,h- Mots d’enfants,C3,Acceptable,Acceptable
8,i- Love & Green,C2,Bon,Bon
9,j- Lotus Baby,C2,Bon,Bon


Taux mauvaise classification Opt 50.0 %
Taux mauvaise classification Pessimiste 50.0 %


### c.Interpretation des resultats pour λ = 55 %

	
Les résultats issus des deux procédures pessimiste et optimiste (Colonne 3 et 4), et les taux de mauvaise classification obtenus montrent que la procédure pessimiste est plus efficace en terme de performance et que la procédure optimiste est plus restrictive.
Ceci est dû au fait que pour pouvoir affecter une couche culotte à une catégorie donnée, il faut que le site soit jugé au moins “aussi performant” que la frontière basse de la catégorie. Ce qui traduit une certaine prudence dans la procédure d’affectation. 

Nous remarquons que : 
les classements proposés par les deux procédures ne sont pas les mêmes quand lambda=0.75, par contre ces derniers sont identiques lorsque lambda=0.55.
La couche jugée la meilleure par les deux procédures est la couche a-Joone, malgré le faite qu’elle soit la plus chère couche-culotte soit 0,40 euros l’unité et qu’elle ne soit pas bio. Ce qui veut dire qu’effectivement ces deux critères prix et bio ne rentrent pas dans la notation fournie par les utilisateurs.

Tandis que la pire couche est Lillydoo selon la procédure pessimiste ainsi que l-Lillydoo et  h-Mots d’enfants d’après la procédure optimiste.
Quant aux couches b-Pamp. Prem, c-Pamp. Baby et d-Naty, ils sont considérés comme bons sans être toutefois meilleurs. 
Les résultats obtenues par Electre tri comparés aux résultats obtenues par le magazine ne sont pas assez identiques donc on peut dire que la manière dont les produits ont été classés par le magazine n’est pas fiable.


## Conclusion


Après l’analyse que nous avons réalisé en utilisant deux méthodes différentes dans les parties précédentes, une méthode par fonction d’utilité (somme pondérée) et une méthode basée sur le surclassement (Electre tri), Nous réalisons que nous obtenons des résultats presque similaires en terme de meilleure couche élue qui est la couche a-Joone.

On déduit que, certes ces deux méthodes sont conçues pour l’analyse multicritères de données mais elles ne répondent pas au même objectif qui est de détecter le meilleur produit ou action pour la méthode de somme pondérée, ou qui est de classifier un ensemble de produits dans le cas de la méthode Electre tri. En terme de classement le classement des produits n’est pas le même pour les deux méthodes.

Parmi les désavantages de la première méthode (Somme pondérée) c’est la compensation des mauvais critères par les critères plus performants. En synthétisant le problème en une fonction unique, on élimine de l'information importante en masquant les critères discriminants. En effet, un mauvais critère pourra toujours être compensé ici par un bon critère. 
Pour la deuxième méthode c’est la multitude des paramètres utilisés (seuil de majorité, poids des critères ..) mais cette dernière reste une méthode très efficace.


In [33]:
%%capture
csv_export='./results_notebook/Couches_Analyse_Classement.xlsx'
exportInExcel(csv_export, 'Electre-tri-Couches',
                 [new_df_lambda1, new_df_lambda_2],
                 ['Modèle lambda 0.55', 'Modèle lambda 0.75'],
                 [result_lambda1, result_lambda_2], 
                 electre = True)